# ETL Process

In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

## Set Paths

In [57]:
logfile = "logfile.txt"
targetfile = "transfomed_data.csv"

## Extract

In [2]:
# CSV Extract function
def extract_from_csv(csvfile):
    dataframe = pd.read_csv(csvfile)
    return dataframe

In [38]:
# JSON Extract function
def extract_from_json(jsonfile):
    dataframe = pd.read_json(jsonfile, lines=True)
    return dataframe

In [31]:
# XML Extract function
def extract_from_xml(xmlfile):
    tree = ET.parse(xmlfile)
    root = tree.getroot()
    cars = list()
    for car in root:
        car_model = car.find("car_model").text
        year_of_manufacture = car.find("year_of_manufacture").text
        price = float(car.find("price").text)
        fuel = car.find("fuel").text
        cars.append({"car_model" : car_model, "year_of_manufacture" : year_of_manufacture, "price" : price, "fuel" : fuel})
    df_cols = list(cars[0].keys())
    dataframe = pd.DataFrame(cars, columns=df_cols)
    return dataframe

In [39]:
# Extract function
def extract():
    extracted_data = pd.DataFrame(columns=["car_model", "year_of_manufacture", "price", "fuel"])

    for csvfile in glob.glob("dealership_data/*.csv"):
        extracted_data = pd.concat([extracted_data, extract_from_csv(csvfile)], ignore_index=True)

    for jsonfile in glob.glob("dealership_data/*.json"):
        extracted_data = pd.concat([extracted_data, extract_from_json(jsonfile)], ignore_index=True)

    for xmlfile in glob.glob("dealership_data/*.xml"):
        extracted_data = pd.concat([extracted_data, extract_from_xml(xmlfile)], ignore_index=True)
        
    return extracted_data

## Transform

In [55]:
# Transform function
def transform(data):
    data["price"] = round(data["price"], 2) # round price column to 2 decimal places
    return data

## Loading

In [71]:
# Load function
def load(targetfile, data_to_load):
    data_to_load.to_csv(targetfile)

# Loging

In [69]:
def log(message):
    timestamp_format = "%d-%m-%Y %H:%M:%S"
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open(logfile, "a") as f:
        f.write(timestamp + "," + message + "\n")

## Running ETL Process

In [72]:
log("ETL process started")
log("Extract phase started")
extracted_data = extract()
log("Extract phase ended")
log("Transfom phase started")
transfomed_data = transform(extracted_data)
log("Transofm phase ended")
log("Load phase started")
load(targetfile, transfomed_data)
log("Load phase ended")
log("ETL process ended")